## Bibliotecas Utilizadas

In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import os
import librosa
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
       print(os.path.join(dirname, filename))



/kaggle/input/zenodo-sounds/emoUERJ/m04s08.wav
/kaggle/input/zenodo-sounds/emoUERJ/m03n08.wav
/kaggle/input/zenodo-sounds/emoUERJ/w02n01.wav
/kaggle/input/zenodo-sounds/emoUERJ/w03a05.wav
/kaggle/input/zenodo-sounds/emoUERJ/m03h04.wav
/kaggle/input/zenodo-sounds/emoUERJ/m01s07.wav
/kaggle/input/zenodo-sounds/emoUERJ/m02a09.wav
/kaggle/input/zenodo-sounds/emoUERJ/m02n05.wav
/kaggle/input/zenodo-sounds/emoUERJ/w01a01.wav
/kaggle/input/zenodo-sounds/emoUERJ/w01s08.wav
/kaggle/input/zenodo-sounds/emoUERJ/w04s07.wav
/kaggle/input/zenodo-sounds/emoUERJ/m03n03.wav
/kaggle/input/zenodo-sounds/emoUERJ/m01n02.wav
/kaggle/input/zenodo-sounds/emoUERJ/w04a16.wav
/kaggle/input/zenodo-sounds/emoUERJ/m01n09.wav
/kaggle/input/zenodo-sounds/emoUERJ/w04s16.wav
/kaggle/input/zenodo-sounds/emoUERJ/w01h10.wav
/kaggle/input/zenodo-sounds/emoUERJ/w04a11.wav
/kaggle/input/zenodo-sounds/emoUERJ/m04n03.wav
/kaggle/input/zenodo-sounds/emoUERJ/w04s08.wav
/kaggle/input/zenodo-sounds/emoUERJ/m02a08.wav
/kaggle/input

## Criação de DATAFRAME 

In [2]:

data_path = "/kaggle/input/zenodo-sounds/emoUERJ"
file_names = os.listdir(data_path)

rows = []
for f in file_names:
    if f.endswith(".wav"):
        # Ex.: f = "m01a01.wav"
        # Remove extensão
        name_without_ext = os.path.splitext(f)[0]  # "m01a01"
        
        # Emoção na posição 4 (índice 3)
        emotion_char = name_without_ext[3]
        
        # Mapeia o caractere para rótulo completo
        if emotion_char == 'h':
            emotion_label = 'happy'
        elif emotion_char == 'a':
            emotion_label = 'angry'
        elif emotion_char == 's':
            emotion_label = 'sad'
        elif emotion_char == 'n':
            emotion_label = 'neutral'
        else:
            emotion_label = 'unknown'
        
        # Caminho completo do arquivo para posterior uso
        full_path = os.path.join(data_path, f)
        
        rows.append([full_path, emotion_label])

# Cria DataFrame
df = pd.DataFrame(rows, columns=["path", "label"])
df.head(10)


,path,label
0,/kaggle/input/zenodo-sounds/emoUERJ/m04s08.wav,sad
1,/kaggle/input/zenodo-sounds/emoUERJ/m03n08.wav,neutral
2,/kaggle/input/zenodo-sounds/emoUERJ/w02n01.wav,neutral
3,/kaggle/input/zenodo-sounds/emoUERJ/w03a05.wav,angry
4,/kaggle/input/zenodo-sounds/emoUERJ/m03h04.wav,happy
5,/kaggle/input/zenodo-sounds/emoUERJ/m01s07.wav,sad
6,/kaggle/input/zenodo-sounds/emoUERJ/m02a09.wav,angry
7,/kaggle/input/zenodo-sounds/emoUERJ/m02n05.wav,neutral
8,/kaggle/input/zenodo-sounds/emoUERJ/w01a01.wav,angry
9,/kaggle/input/zenodo-sounds/emoUERJ/w01s08.wav,sad


## Salvando o DataFrame 

In [3]:
df.to_csv("metadata.csv", index=False)


## Treino e Validação 

In [4]:
from sklearn.model_selection import train_test_split

train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)

print("Tamanho treino:", len(train_df))
print("Tamanho validação:", len(val_df))


Tamanho treino: 301
Tamanho validação: 76


## Carregamento do Modelo

In [5]:
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2ForSequenceClassification

model_name = "facebook/wav2vec2-large-xlsr-53"  # Exemplo
feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(model_name)

# Defina o número de classes baseado no seu DF
unique_labels = sorted(df['label'].unique().tolist())
label2id = {label: i for i, label in enumerate(unique_labels)}
id2label = {i: label for label, i in label2id.items()}
num_labels = len(unique_labels)

model = Wav2Vec2ForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label
)


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Criação de Datasets a partir de DataFrames

In [6]:
from datasets import Dataset

train_dataset_ds = Dataset.from_pandas(train_df)
val_dataset_ds   = Dataset.from_pandas(val_df)


## Extração de Características

In [7]:

def preprocess_function(examples):
    audio_path = examples["path"]
    # Carrega o áudio
    waveform, sr = librosa.load(audio_path, sr=16000, mono=True)
    
    # Extrai features
    inputs = feature_extractor(waveform, sampling_rate=16000, return_tensors="pt", padding=True)
    
    # Se seu dataset tiver "label", converta em índice
    label_str = examples["label"]
    label_id = label2id[label_str]
    
    # Monta o dicionário de saída
    out = {
        "input_values": inputs["input_values"][0], 
        "labels": label_id
    }
    # Se o feature_extractor retornar "attention_mask", você pode incluir aqui:
    if "attention_mask" in inputs:
        out["attention_mask"] = inputs["attention_mask"][0]
    
    return out


## função de pré-processamento a cada exemplo dos datasets de treino e validação, transformando os dados brutos para o formato necessário ao modelo

In [8]:
train_dataset_ds = train_dataset_ds.map(preprocess_function)
val_dataset_ds   = val_dataset_ds.map(preprocess_function)                                             


Map:   0%|          | 0/301 [00:00<?, ? examples/s]

Map:   0%|          | 0/76 [00:00<?, ? examples/s]

## Início de treinamento do modelo 

In [9]:
from transformers import TrainingArguments, Trainer
import numpy as np
from sklearn.metrics import accuracy_score, f1_score


# Aplicar a função de pré-processamento aos datasets
train_dataset_ds = train_dataset_ds.map(preprocess_function)
val_dataset_ds   = val_dataset_ds.map(preprocess_function)

# Remover as colunas que não são necessárias (como "path" e "label")
train_dataset_ds = train_dataset_ds.remove_columns(["path", "label"])
val_dataset_ds   = val_dataset_ds.remove_columns(["path", "label"])


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="macro")
    return {"accuracy": acc, "f1": f1}

training_args = TrainingArguments(
    output_dir="wav2vec2_finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=9,
    per_device_train_batch_size=4,
    learning_rate=1e-4,
    logging_steps=10,
    report_to=[],
    save_total_limit=1  # mantém apenas o checkpoint mais recente
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset_ds,
    eval_dataset=val_dataset_ds,
    tokenizer=feature_extractor,  # se usar 'trainer.predict'
    compute_metrics=compute_metrics
)

trainer.train()


Map:   0%|          | 0/301 [00:00<?, ? examples/s]

Map:   0%|          | 0/76 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-9-aecd2293aafd>:35: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.373600,1.299542,0.513158,0.478643
2,1.193300,0.986820,0.605263,0.526798
3,0.905200,0.806108,0.605263,0.549567
4,0.690700,0.806955,0.539474,0.430460
5,0.740900,0.845383,0.618421,0.571713
6,0.627400,0.807916,0.671053,0.644819
7,0.612800,0.517300,0.789474,0.782329
8,0.460800,0.500408,0.828947,0.817804
9,0.513800,0.521481,0.842105,0.836291


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

TrainOutput(global_step=342, training_loss=0.8045652470393487, metrics={'train_runtime': 591.8201, 'train_samples_per_second': 4.577, 'train_steps_per_second': 0.578, 'total_flos': 4.01268417805559e+17, 'train_loss': 0.8045652470393487, 'epoch': 9.0})

In [10]:
eval_metrics = trainer.evaluate()
print(eval_metrics)
  

/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


{'eval_loss': 0.5214807987213135, 'eval_accuracy': 0.8421052631578947, 'eval_f1': 0.8362914862914862, 'eval_runtime': 6.1456, 'eval_samples_per_second': 12.367, 'eval_steps_per_second': 0.814, 'epoch': 9.0}


## Testando o Modelo Áudio único Locutor 

In [11]:
def predict_emotion(audio_path):
    # Carrega o áudio com 16kHz e mono
    audio, sr = librosa.load(audio_path, sr=16000, mono=True)
    
    # Extrai features com truncation e padding, definindo max_length (ex: 16000 amostras = 1s)
    inputs = feature_extractor(
        audio,
        sampling_rate=sr,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=16000
    )
    
    # Mova os tensores para o mesmo dispositivo do modelo (ex.: cuda:0)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        pred_id = torch.argmax(logits, dim=-1).item()
    
    return id2label[pred_id]

# Exemplo de uso:
audio_test_path = "/kaggle/input/zenodo-sounds/emoUERJ/w03a04.wav"   
predicted_emotion = predict_emotion(audio_test_path)
print("Emoção detectada:", predicted_emotion)


Emoção detectada: happy


## Testando o Modelo Áudio mais de um  Locutor 

In [12]:
!pip install pyannote.audio huggingface_hub

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 818.9/818.9 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.5/58.5 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.9/125.9 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.7/117.7 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━